# Doc vectors

In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
from cord.jsonpaper import load_json_cache
from cord.core import JSON_CATALOGS, BIORXIV_MEDRXIV, NONCOMM_USE_SUBSET, COMM_USE_SUBSET
import nltk
from sklearn.feature_extraction.text import HashingVectorizer

In [21]:
bio = load_json_cache(BIORXIV_MEDRXIV)

Loading json cache files for biorxiv_medrxiv


In [27]:
def get_token_vector(index_tokens):
    vectorizer = HashingVectorizer(n_features=2**4)
    sentences = index_tokens.apply(lambda t: ' '.join(t))
    token_vectors = vectorizer.fit_transform(sentences)
    print(token_vectors.shape)
    return token_vectors
    
%time bio['token_vector'] = get_token_vector(bio.index_tokens)

(1342, 16)
Wall time: 1.71 s


In [38]:
bio.loc[0].token_vector.shape

(1342, 16)

In [30]:
from annoy import AnnoyIndex

In [32]:
index = AnnoyIndex(16, 'angular')

In [ ]:
from gensim.models import Sent2Vec
from gensim.test.utils import common_texts

model = Sent2Vec(common_texts, size=100, min_count=1)
model.build_vocab(bio.index_tokens, update=True)
model.train(bio.index_tokens, total_examples=model.corpus_count, epochs=model.epochs)

In [44]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(bio.index_tokens)]
model = Doc2Vec(documents, vector_size=20, window=2, min_count=1, workers=4)